In [55]:
import numpy as np
from matplotlib import pyplot as plt

In [5]:
from HW1 import *

genes = readFile("Assignment1Sequences.txt")

# Initialize the lookup table
AATable = setUpTable()

# Holds the resulting genes in AA format
AASeqs = {}
for gene in genes:
    # Convert RNA string to AA format and store in the
    # AASeqs dictionary
    AASeqs[gene] = (convertToAAs(genes[gene], AATable, single_letter=False))

In [6]:
gene1 = AASeqs['Gene1']
gene2 = AASeqs['Gene2']
gene3 = AASeqs['Gene3']

In [68]:
# gene1_domains = [(257, 279), (328, 350), (370, 392), (455,477), (509, 531)]
# gene2_domains = []
# gene3_domains = [(647, 669)]


gene1_domains = [(257, 275), (327, 348), (372, 390), (427, 445), (455, 473), (510, 531)]

In [91]:
hw_hydrophobicity = {   
    'Ala': -0.5,
    'Arg': 3.0,
    'Asn': 0.2,
    'Asp': 3.0,
    'Cys': -1.0,
    'Gln': 0.2,
    'Glu': 3.0,
    'Gly': 0.0,
    'His': -0.5,
    'Ile': -0.8,
    'Leu': -1.8,
    'Lys': -1.3,
    'Met': 3.0,
    'Phe': -2.5,
    'Pro': 0.0,
    'Ser': 0.3,
    'Thr': -0.4,
    'Trp': -3.4,
    'Tyr': -2.3,
    'Val': -1.5,
    '*': 0
}

hydropathy = {   
    'Ala': 1.8,
    'Arg': -4.5,
    'Asn': -3.5,
    'Asp': -3.5,
    'Cys': 2.5,
    'Gln': -3.5,
    'Glu': -3.5,
    'Gly': -0.4,
    'His': -3.2,
    'Ile': 4.5,
    'Leu': 3.8,
    'Lys': -3.9,
    'Met': 1.9,
    'Phe': 2.8,
    'Pro': -1.6,
    'Ser': -0.8,
    'Thr': 0.7,
    'Trp': -0.9,
    'Tyr': -1.3,
    'Val': -4.2,
    '*': 0
}

polarity = {   
    'Ala': 'N',
    'Arg': '+',
    'Asn': 'P',
    'Asp': '-',
    'Cys': 'P',
    'Gln': 'P',
    'Glu': '-',
    'Gly': 'N',
    'His': '+',
    'Ile': 'N',
    'Leu': 'N',
    'Lys': '+',
    'Met': 'N',
    'Phe': 'N',
    'Pro': 'N',
    'Ser': 'P',
    'Thr': 'P',
    'Trp': 'N',
    'Tyr': 'P',
    'Val': 'N',
    '*': 0
}

triple_letter_to_single_letter = {   
    'Ala': 'A',
    'Arg': 'R',
    'Asn': 'N',
    'Asp': 'D',
    'Asx': 'B',
    'Cys': 'C',
    'Gln': 'Q',
    'Glu': 'E',
    'Glx': 'Z',
    'Gly': 'G',
    'His': 'H',
    'Ile': 'I',
    'Leu': 'L',
    'Lys': 'K',
    'Met': 'M',
    'Phe': 'F',
    'Pro': 'P',
    'Ser': 'S',
    'Thr': 'T',
    'Trp': 'W',
    'Tyr': 'Y',
    'Val': 'V'
}

name_to_tripe_letter = {
    'alanine': 'ala',
    'arginine': 'arg',
    'asparagine': 'asn',
    'asparagine or aspartic acid': 'asx',
    'aspartic acid': 'asp',
    'cysteine': 'cys',
    'glutamic acid': 'glu',
    'glutamine': 'gln',
    'glutamine or glutamic acid': 'glx',
    'glycine': 'gly',
    'histidine': 'his',
    'isoleucine': 'ile',
    'leucine': 'leu',
    'lysine': 'lys',
    'methionine': 'met',
    'phenylalanine': 'phe',
    'proline': 'pro',
    'serine': 'ser',
    'threonine': 'thr',
    'tryptophan': 'trp',
    'tyrosine': 'tyr',
    'valine': 'val'
}

# Interface scale, Octanol scale, Octanol interface
# https://en.wikipedia.org/wiki/Hydrophobicity_scales
hydrophobic_scales = {   
    'Ala': [0.17, 0.5, 0.33],
    'Arg': [0.81, 1.81, 1.0],
    'Asn': [0.42, 0.85, 0.43],
    'Asp-': [1.23, 3.64, 2.41],
    'Asp': [-0.07, 0.43, 0.5],
    'Cys': [-0.24, -0.02, 0.22],
    'Gln': [0.58, 0.77, 0.19],
    'Glu-': [2.02, 3.63, 1.61],
    'Glu': [-0.01, 0.11, 0.12],
    'Gly': [0.01, 1.15, 1.14],
    'His+': [0.96, 2.33, 1.37],
    'His': [0.17, 0.11, -0.06],
    'Ile': [-0.31, -1.12, -0.81],
    'Leu': [-0.56, -1.25, -0.69],
    'Lys': [0.99, 2.8, 1.81],
    'Met': [-0.23, -0.67, -0.44],
    'Phe': [-1.13, -1.71, -0.58],
    'Pro': [0.45, 0.14, -0.31],
    'Ser': [0.13, 0.46, 0.33],
    'Thr': [0.14, 0.25, 0.11],
    'Trp': [-1.85, -2.09, -0.24],
    'Tyr': [-0.94, -0.71, 0.23],
    'Val': [0.07, -0.46, -0.53],
    '*'  : [0, 0, 0]
}

In [40]:
for a, b in gene1_domains:
    r = []
    for i in range(a-1, b):
        r.append('{}'.format(polarity[gene1[i]]))
    print('Helix from {}-{}:'.format(a, b), ' '.join(r))

Helix from 257-279: N N - N N N P N N N N N N N P N N N P P N P N
Helix from 328-350: N N N N N N N N N N P N P N N N P N N P N N +
Helix from 370-392: N N P P N N N P N N N N N P N N N N N N N N P
Helix from 455-477: N N N N N N N N N N N N N P N P N N P P P N P
Helix from 509-531: N N N N N N N N N N N N N N N N N N P - N P N


In [41]:
for a, b in gene3_domains:
    r = []
    for i in range(a-1, b):
        r.append('{}'.format(polarity[gene3[i]]))
    print('Helix from {}-{}:'.format(a, b), ' '.join(r))

Helix from 647-669: N + P N N N N N + N N N P N N N N N N P P N N


In [50]:
for a, b in gene1_domains:
    r = []
    v = []
    for i in range(a-1, b):
        r.append('{:.1f}'.format(hw_hydrophobicity[gene1[i]]))
        v.append(hw_hydrophobicity[gene1[i]])
    print('Helix from {}-{}:'.format(a, b), ', '.join(r))
    print('Mean: {}'.format(np.mean(v)))

Helix from 257-279: -1.8, -0.5, 3.0, -2.5, 3.0, 0.0, -0.4, 3.0, -1.5, 3.0, -0.8, -0.8, -2.5, 0.0, 0.3, -0.5, -1.5, -1.5, -1.0, 0.2, -1.5, 0.2, -1.5
Mean: -0.24347826086956517
Helix from 328-350: -1.8, 0.0, -3.4, -0.5, -0.5, -0.5, -1.5, -1.5, 3.0, 0.0, -2.3, -2.5, -1.0, -0.5, 0.0, 0.0, 0.3, -0.5, -0.8, 0.3, 0.0, -0.5, -0.5
Mean: -0.6391304347826087
Helix from 370-392: -1.5, 0.0, -2.3, -2.3, -2.5, -0.5, 0.0, 0.2, -1.8, -0.8, 0.0, -0.5, -2.5, -0.4, 0.0, -0.5, -1.8, -0.8, -1.8, -2.5, -0.8, -3.4, -2.3
Mean: -1.2521739130434784
Helix from 455-477: -1.5, -2.5, 0.0, -1.8, 3.0, 3.0, -2.5, -0.8, -1.8, -0.8, -2.5, -0.8, -0.8, 0.2, -0.5, 0.3, 3.0, -0.5, -2.3, 0.2, -0.4, 0.0, -0.4
Mean: -0.4434782608695652
Helix from 509-531: -2.5, -3.4, -1.5, 0.0, 3.0, -1.5, 0.0, 0.0, -2.5, -0.8, 0.0, -0.5, -1.8, 3.0, 0.0, 0.0, -1.8, -1.5, -2.3, 3.0, -1.5, -1.0, -0.8
Mean: -0.6260869565217392


In [51]:
for a, b in gene3_domains:
    r = []
    v = []
    for i in range(a-1, b):
        r.append('{:.1f}'.format(hw_hydrophobicity[gene3[i]]))
        v.append(hw_hydrophobicity[gene3[i]])
    print('Helix from {}-{}:'.format(a, b), ', '.join(r))
    print('Mean: {}'.format(np.mean(v)))

Helix from 647-669: -1.5, -0.5, -0.4, 3.0, -2.5, -1.8, -1.5, -1.8, -1.3, 0.0, -1.8, -1.8, 0.3, 3.0, -0.5, 0.0, -0.5, -1.8, -1.5, -1.0, 0.3, -1.8, -1.5
Mean: -0.7347826086956522


In [43]:
for a, b in gene1_domains:
    r = []
    for i in range(a-30, a):
        r.append('{}'.format(polarity[gene1[i]]))
    print('Helix from {}-{}:'.format(a-10, a), ' '.join(r))
    
    
print()

for a, b in gene1_domains:
    r = []
    for i in range(a-30, a):
        r.append('{}'.format(triple_letter_to_single_letter[gene1[i]]))
    print('Helix from {}-{}:'.format(a-10, a), ' '.join(r))

Helix from 247-257: P N P P N P N N N P P P + N N P + N P P N + P P P N + - N N
Helix from 318-328: N P P N - P N - N N + P N P P N N P P N N N N P N - - N N N
Helix from 360-370: N N N P N N P N N + N P N P N P N N P N N P + N N N N + + N
Helix from 445-455: P N N P - N N P N N N N P N N P N N N P - N P P P N P P - N
Helix from 499-509: P N N + - N N N + N N N P N N N N - + + N N N N + + + + N N

Helix from 247-257: N P Q T P T V L P S T Y H P I N K W S S V K N T Y L K E F L
Helix from 318-328: G S S A E T I D A M K S L T S L V S S V A G G T F D D V A L
Helix from 360-370: A G G S A I S G A H L N P S I T L A N L V Y R G F P L K K V
Helix from 445-455: Q F F S E F L C G A M L Q A G T F A L T D P Y T C L S S D V
Helix from 499-509: N L A R D L G P R L A L Y A V G F D H K M L W V H H H H F F


In [33]:
for a, b in gene1_domains:
    r = []
    for i in range(b+1, b+30):
        r.append('{}'.format(polarity[gene1[i]]))
    print('Seq from {}-{}:'.format(b, b+30), ' '.join(r))
    
print()

for a, b in gene1_domains:
    r = []
    for i in range(b+1, b+30):
        r.append('{}'.format(triple_letter_to_single_letter[gene1[i]]))
    print('Seq from {}-{}:'.format(a-10, a), ' '.join(r))

Seq from 279-309: N + N P P - P N P N N N - P N P N P N P P N - P N - N N +
Seq from 350-380: P N P N P N N P N N P + N N N N + + N N P P N N N P N N N
Seq from 392-422: + N N P - N P P - N N N P - P N N N N N P N N N + N P N P
Seq from 477-507: N P N N + - N N N + N N N P N N N N - + + N N N N + + + +
Seq from 531-561: P N + - P N N P N P N N N P + - N N N + N N N + + N N N +

Seq from 247-257: G K I Q Q D N F N V A L D N L N V T G S S A E T I D A M K
Seq from 318-328: N P S I T L A N L V Y R G F P L K K V P Y Y F A G Q L I G
Seq from 360-370: R V L Q E A Y S D W W M N E S V A G M F C V F P K P Y L S
Seq from 445-455: M N L A R D L G P R L A L Y A V G F D H K M L W V H H H H
Seq from 499-509: Q G H E S P V N W S L P V Y K E M I M R A W F R R P G W K


In [38]:
for a, b in gene1_domains:
    r = []
    for i in range(a-10, b+10):
        r.append('{}'.format(triple_letter_to_single_letter[gene1[i]] if gene1[i] in ['Lys', 'Arg'] else '.'))
    r = r[:10] + ['|'] + r[10:31] + ['|'] + r[31:]
    print('{}-{}:'.format(b, b+30), ' '.join(r))

279-309: . K . . . . K . . . | . . . . . . . . . . . . . . . . . . . . . | . . . K . . . . . . .
350-380: . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . | . . . . . . . . . . .
392-422: . . R . . . . K K . | . . . . . . . . . . . . . . . . . . . . . | . . R . . . . . . . .
477-507: . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . | . . . . . . R . . . .
531-561: . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . | . . . . . . . . . . .


In [92]:

hydrop_vals = [hydropathy[aa] for aa in gene1]

window_length = 20

some1 = [0 for i in range(len(hydrop_vals) - window_length + 1)]
val = 0

for i in range(window_length):
    if (hydrop_vals[i]):
        val += hydrop_vals[i]
        
some1[0] = val

for i in range(1, len(hydrop_vals) - window_length + 1):
    val -= hydrop_vals[i-1] if hydrop_vals[i-1] else 0
    val += hydrop_vals[i + 20 -1] if hydrop_vals[i + 20 -1] else 0
    some1[i] = val

some1 = np.array(some1)/20
print(some1, len(some1))

[ -1.10000000e+00  -1.37000000e+00  -1.37000000e+00  -1.23500000e+00
  -1.38000000e+00  -1.40000000e+00  -1.38000000e+00  -1.51000000e+00
  -1.87500000e+00  -1.89500000e+00  -1.89500000e+00  -2.07500000e+00
  -2.30500000e+00  -2.44000000e+00  -2.57500000e+00  -2.42000000e+00
  -2.60500000e+00  -2.43500000e+00  -2.31500000e+00  -2.39000000e+00
  -2.40500000e+00  -2.31000000e+00  -2.31000000e+00  -2.43000000e+00
  -2.36500000e+00  -2.33000000e+00  -2.19500000e+00  -2.17500000e+00
  -2.02000000e+00  -1.79000000e+00  -1.79000000e+00  -1.92500000e+00
  -1.92500000e+00  -1.92500000e+00  -1.92500000e+00  -2.08000000e+00
  -2.03000000e+00  -2.16500000e+00  -2.30000000e+00  -2.43500000e+00
  -2.43500000e+00  -2.53000000e+00  -2.53000000e+00  -2.54500000e+00
  -2.56000000e+00  -2.57500000e+00  -2.71000000e+00  -2.86500000e+00
  -2.86500000e+00  -3.07500000e+00  -3.07500000e+00  -2.92000000e+00
  -2.90000000e+00  -2.90000000e+00  -2.90000000e+00  -2.90000000e+00
  -2.90000000e+00  -2.79000000e+00

In [80]:
some1 = [hydrophobic_scales[aa][0] for aa in gene1]

In [97]:
%matplotlib
plt.plot(some1, c='lightblue')
plt.axhline(1.6, 0, 670)
for a, b in gene1_domains:
    plt.axvline(a-1, 0, 1, c='r')
    plt.axvline(b, 0, 1, c='g')

Using matplotlib backend: MacOSX


In [63]:

hydrop_vals = [hw_hydrophobicity[aa] for aa in gene3]

window_length = 20

some3 = [0 for i in range(len(hydrop_vals) - window_length + 1)]
val = 0

for i in range(window_length):
    if (hydrop_vals[i]):
        val += hydrop_vals[i]
        
some3[0] = val

for i in range(1, len(hydrop_vals) - window_length + 1):
    val -= hydrop_vals[i-1] if hydrop_vals[i-1] else 0
    val += hydrop_vals[i + 20 -1] if hydrop_vals[i + 20 -1] else 0
    some3[i] = val

some3 = np.array(some3)/20
print(some3, len(some3))

[  8.55000000e-01   6.80000000e-01   6.85000000e-01   6.70000000e-01
   5.65000000e-01   5.65000000e-01   5.15000000e-01   5.65000000e-01
   5.75000000e-01   5.75000000e-01   6.30000000e-01   6.45000000e-01
   4.70000000e-01   2.30000000e-01   5.50000000e-02  -1.20000000e-01
  -2.95000000e-01  -4.65000000e-01  -4.65000000e-01  -4.65000000e-01
  -4.40000000e-01  -4.15000000e-01  -3.95000000e-01  -3.95000000e-01
  -3.05000000e-01  -3.55000000e-01  -1.30000000e-01  -1.30000000e-01
  -5.00000000e-02  -5.00000000e-02  -5.00000000e-02  -1.00000000e-01
  -6.00000000e-02   5.00000000e-03  -4.50000000e-02  -4.50000000e-02
   1.30000000e-01   1.30000000e-01   1.50000000e-01   3.25000000e-01
   3.40000000e-01   3.55000000e-01   2.80000000e-01   2.80000000e-01
   2.30000000e-01   3.20000000e-01   8.00000000e-02   1.05000000e-01
   1.00000000e-01   1.00000000e-01   6.00000000e-02   6.00000000e-02
   2.00000000e-02   1.95000000e-01   4.20000000e-01   4.25000000e-01
   2.75000000e-01   3.05000000e-01

In [95]:
%matplotlib
plt.plot(some3)
plt.axhline(1.6, 0, 670)
for a, b in gene3_domains:
    plt.axvline(a-1, 0, 1, c='r')
    plt.axvline(b, 0, 1, c='g')

Using matplotlib backend: MacOSX


In [90]:
import pprint
